'''1. Para desarrollar este desafío necesitarás los siguientes archivos:
● incidents.pkl
● officers.pkl
● subjects.pkl
a. Carga los datos y crea un DataFrame con cada uno de ellos.
b. Genera una tabla que contenga la unión de las 3 tablas. hint: utiliza sufijos para para las
columnas que se llaman igual usando el parámetro suffixes de pd.merge().
c. Verifica si hay filas duplicadas; si es así, elimínalas.
d. ¿Cuántos sujetos de género F hay en el DataFrame resultante? hint: usa el método
.value_counts() sobre la columna.
e. ¿En cuántos números de caso hay por lo menos una sospechosa que sea mujer? hint:
utiliza el método unique() para obtener los valores únicos de una columna específica de
un DataFrame luego de filtrar.
f. Genera una tabla pivote que muestre el número de casos por mes en la fila y por año en
las columnas. hint: genere columnas de tipo fecha con pd.to_datetime(). ¿Se ve alguna
relación entre mes del año y cantidad de casos?
g. Genera una tabla pivote que muestre en las filas el género del oficial y en las columnas el
género del subject. ¿Cómo interpretas los valores que muestra esta vista?'''


In [1]:
import pandas as pd #FernandoSanchez
import numpy as np

# cargamos los dataframes
incident_df = pd.read_pickle('incidents.pkl') #FernandoSanchez
officer_df = pd.read_pickle('officers.pkl')
subject_df = pd.read_pickle('subjects.pkl')
# generamos una tabla que contenga la union de las tres tablas
merged_df = pd.merge(pd.merge(incident_df, officer_df, on='case_number', suffixes=('_incident', '_officer')), subject_df, on='case_number', 
                     suffixes=('_officer', '_subject')) #FernandoSanchez sufixes lo usamos para poner especificamente los sufijos para identificar las columnas
# verificamos si hay filas duplicadas
merged_df = merged_df.drop_duplicates() #FernandoSanchez aqui eliminamos los duplicados del merge
print(merged_df.columns)
# obtenemos el numero de sujetos de genero femenino
f_count_subject = merged_df['gender_subject'].value_counts().get('F', 0)
print(f"Número de sujetos de género F: {f_count_subject}")
# obtenemos el numero de incidentes por lo menos con una sospechosa que sea mujer
f_sospecha = merged_df[merged_df['gender_subject'] == 'F']['case_number'].unique() #FernandoSanchez unique sirve para obtener valores unicos
print(f"Número de casos con al menos una sospechosa de que es mujer: {f_sospecha}")
'''FernandoSanchez aquí hacemos el conteo del genero femenino se pone con _subject para eso los sufijos ya que tenemos
dos columnas con el genero ya que venian en dos dataframes una de oficial y otra de sujeto'''
# generamos la tabla pivote para casos por mes y año
merged_df['date'] = pd.to_datetime(merged_df['date']) #FernandoSanchez to_datetime es para cambiar a tipo de dato fecha y hora 
pivot_mesyano = pd.pivot_table(merged_df, values='case_number', 
                                           index=merged_df['date'].dt.month, #FernandoSanchez aqui apareceran los meses como indice
                                           columns=merged_df['date'].dt.year, #FernandoSanchez aquí ponemos como columna cada año
                                           aggfunc='count', fill_value=0) #aquí hacemos el conteo de casos
print("\nTabla pivote por mes y año:")
print(pivot_mesyano)
# generamos la tabla pivote de genero del oficial y columna el del sujeto
pivot_gender = pd.pivot_table(merged_df, values='case_number', 
                                    index='gender_officer', #FernandoSanchez aqui apareceran los oficiales como indice
                                    columns='gender_subject', #FernandoSanchez aquí ponemos como columna los sujetos
                                    aggfunc='count', fill_value=0)#aquí hacemos el conteo de casos

print("\nTabla pivote por género del oficial y del sujeto:")
print(pivot_gender)
'''Se logra ver que los mayores casos son hombre_oficial con hombre_sujeto, los menos mujer_oficial con mujer_sujeto y mas casos
de mujer_oficial con hombre_sujeto que hombre_oficial con mujer_sujeto '''

Index(['case_number', 'date', 'location', 'subject_statuses', 'subject_weapon',
       'subjects', 'subject_count', 'officers', 'officer_count',
       'grand_jury_disposition', 'attorney_general_forms_url', 'summary_url',
       'summary_text', 'latitude', 'longitude', 'race_officer',
       'gender_officer', 'last_name_officer', 'first_name_officer',
       'full_name_officer', 'race_subject', 'gender_subject',
       'last_name_subject', 'first_name_subject', 'full_name_subject'],
      dtype='object')
Número de sujetos de género F: 9
Número de casos con al menos una sospechosa de que es mujer: ['106454R' '8025N' '266130-2015' '344699X' '143473-2014' '129379Z'
 '132104Z']

Tabla pivote por mes y año:
date  2003  2004  2005  2006  2007  2008  2009  2010  2011  2012  2013  2014   
date                                                                           
1        2     2     0     1     1     1     2     0     0     0     7     4  \
2        0     1     1     7     1     0     0 

'Se logra ver que los mayores casos son hombre_oficial con hombre_sujeto, los menos mujer_oficial con mujer_sujeto y mas casos\nde mujer_oficial con hombre_sujeto que hombre_oficial con mujer_sujeto '

'''2. Para continuar con el desarrollo de este desafío, necesitarás el archivos
Cleaned_DS_Jobs.csv
a. Carga los datos y crea un DataFrame con ellos.
b. Utiliza la siguiente lista de valores que serán considerados como nulos:
["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]
(hint: utiliza el método replace para reemplazar los valores indicados por np.nan)
c. Elimina todas las filas con datos faltantes. (hint: utiliza el método .dropna())
d. A partir de la columna “Salary Estimate”, genera dos columnas: Salario Estimado Mínimo
y Máximo. (hint: Utiliza el método apply sobre la columna.)
e. Realiza la recodificación de la columna Size con los valores de la siguiente tabla: (hint:
utilice reemplazo con diccionario usando el método replace sobre la columna.)
f. Finalmente, genera una tabla pivote que muestre la media del salario estimado mínimo y
la media del salario estimado máximo por tamaño de empresa. (hint: utiliza
pd.pivot_table para generar la vista adecuada con las columnas generadas.)
'''

In [2]:
# cargamos los datos
data = pd.read_csv("Cleaned_DS_Jobs.csv") #FernandoSanchez
# reemplazamos los valores nulos o vacios
for value in ["na","NA","-1","0","-1","null","n/a","N/A","NULL"]:
    data[data == value] = np.nan #FernandoSanchez con el for buscamos los valores indicados y los cambiamos por lo np.nan
# eliminamos las filas con datos faltantes
data = data.dropna() #quitamos los datos que les faltan cosas
# generamos las columnas Salario Estimado Minimo y Maximo
def minmaxsalary(row): #FernandoSanchez definimos una funcion que separec los salarios por cada tamaño y saque sus minimos y maximos 
    if row['Size']=='1 to 50 employees':
        return [row['min_salary'],row['max_salary']]
    elif row['Size']=='51 to 200 employees':
        return [row['min_salary'],row['max_salary']]
    elif row['Size']=='201 to 500 employees':
        return [row['min_salary'],row['max_salary']]
    elif row['Size']=='501 to 1000 employees':
        return [row['min_salary'],row['max_salary']]
    elif row['Size']=='1001 to 5000 employees':
        return [row['min_salary'],row['max_salary']]
    elif row['Size']=='5001 to 10000 employees':
        return [row['min_salary'],row['max_salary']]
    else:
        return [row['min_salary'],row['max_salary']]
data['SalarioEstimado'] = data.apply(minmaxsalary,axis=1)
# reemplazamos los valores de size en la columna size
size_recode = {
    '1 to 50 employees': 'Pequeña',
    '51 to 200 employees': 'Pequeña',
    '201 to 500 employees': 'Mediana',
    '501 to 1000 employees': 'Mediana',
    '1001 to 5000 employees': 'Grande',
    '5001 to 10000 employees': 'Grande',
    '10000+ employees': 'Muy Grande'
}#FernandoSanchez creamos un diccionario con los valores a cambiar y los que pondremos
data['Size'] = data['Size'].replace(size_recode) #FernandoSanchez utilizamos el metodo replace con el diccionario y cambiamos valores
# generamos la tabla pivot
pivot_salario = pd.pivot_table(data, values=['min_salary', 'max_salary'],#FernandoSanchez indicamos los valores y la funcion maxima para ambos valores separados por el tamaño de empresa
                              index='Size', aggfunc={'min_salary': 'mean', 'max_salary': 'mean'})
print("\nTabla Pivot Salarios: ")
print(pivot_salario)


Tabla Pivot Salarios: 
            max_salary  min_salary
Size                              
Grande      144.411765   97.352941
Mediana     143.935484   97.193548
Muy Grande  151.111111   97.888889
Pequeña     137.666667  100.666667
Unknown     110.500000   73.000000
